<a href="https://colab.research.google.com/github/lawanya-khawate/Projects/blob/main/CS_Deepfake_detector_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import gradio as gr
import os
from sklearn.model_selection import train_test_split


In [ ]:
# Creating synthetic dataset for fake and real videos (images)
def create_dataset(n_samples=500, img_size=64):
    X = []
    y = []

    for _ in range(n_samples):
        # Real images: smooth patterns
        img_real = np.ones((img_size, img_size, 3)) * np.random.uniform(0.7, 1.0)
        X.append(img_real)
        y.append(0)  # Real label

        # Fake images: noisy patterns
        img_fake = np.random.rand(img_size, img_size, 3)
        X.append(img_fake)
        y.append(1)  # Fake label

    return np.array(X), np.array(y)

# Create dataset
X, y = create_dataset()

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Create a simple CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # Output: 1 neuron for Fake/Real
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 289ms/step - accuracy: 0.7313 - loss: 0.5478 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 160ms/step - accuracy: 1.0000 - loss: 5.6051e-04 - val_accuracy: 1.0000 - val_loss: 7.0115e-05
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 189ms/step - accuracy: 1.0000 - loss: 6.6776e-05 - val_accuracy: 1.0000 - val_loss: 4.5423e-05
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - accuracy: 1.0000 - loss: 5.2295e-05 - val_accuracy: 1.0000 - val_loss: 3.8776e-05
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 159ms/step - accuracy: 1.0000 - loss: 4.7575e-05 - val_accuracy: 1.0000 - val_loss: 3.3991e-05


In [ ]:
model.save("deepfake_detector.h5")


In [ ]:
# Reload model
model = tf.keras.models.load_model("deepfake_detector.h5")

# Prediction function
def predict_deepfake(img):
    img = tf.image.resize(img, (64, 64))
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)[0][0]

    if prediction > 0.5:
        label = "🚨 FAKE Video Detected!"
        confidence = prediction * 100
    else:
        label = "✅ REAL Video Detected!"
        confidence = (1 - prediction) * 100

    return f"{label}\nConfidence: {confidence:.2f}%"

# Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("<h1 style='text-align: center; color: blue;'>Deepfake Video Detector 🔍</h1>")
    gr.Markdown("Upload a screenshot/frame of your video 📹, and check if it's FAKE or REAL using our AI model 🚀.")

    with gr.Row():
        image_input = gr.Image(type="numpy", label="Upload Video Frame (Image)")
        output_label = gr.Textbox(label="Prediction Result")

    predict_button = gr.Button("Detect Now 🚀")

    predict_button.click(fn=predict_deepfake, inputs=image_input, outputs=output_label)

# Launch app
app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bde803ef0ee0c2e583.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
